##### Copyright 2018 The TensorFlow Probability Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Explicación de las formas de TensorFlow Distributions

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/probability/examples/Understanding_TensorFlow_Distributions_Shapes"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/probability/examples/Understanding_TensorFlow_Distributions_Shapes.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/probability/examples/Understanding_TensorFlow_Distributions_Shapes.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/probability/examples/Understanding_TensorFlow_Distributions_Shapes.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

In [ ]:
import collections

import tensorflow as tf
tf.compat.v2.enable_v2_behavior()

import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

## Conceptos básicos

Hay tres conceptos importantes asociados con las formas de TensorFlow Distributions:

- La *forma de evento* describe la forma de una única extracción de la distribución; la extracción puede depender de todas las dimensiones. Para distribuciones escalares, la forma de evento es `[]`. Para una MultivariateNormal de 5 dimensiones, la forma de evento es `[5]`.
- La *forma del lote* describe extracciones independientes, no distribuidas de manera idéntica, también conocidas como un "lote" de distribuciones.
- La *forma de muestra* describe extracciones independientes y distribuidas de forma idéntica de lotes de la familia de distribución.

La forma de evento y la forma de lote son propiedades de un objeto `Distribution`, mientras que la forma de muestra se asocia con una llamada específica a `sample` o `log_prob`.

El propósito de este bloc de notas es ilustrar estos conceptos a través de ejemplos, por lo tanto, si no le resulta evidente a primera vista, no se preocupe.

Para obtener otra descripción general conceptual de estos conceptos, consulte [esta publicación de blog](https://ericmjl.github.io/blog/2019/5/29/reasoning-about-shapes-and-probability-distributions/).

### Una nota sobre TensorFlow Eager

Todo este bloc de notas se escribió con [TensorFlow Eager](https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html). Ninguno de los conceptos que se presentan *depende* de Eager, aunque con Eager, las formas de eventos y lotes de distribución se evalúan (y por lo tanto se conocen) cuando el objeto `Distribution` se crea en Python, mientras que en el modo de grafo (no Eager), es posible definir distribuciones cuyas formas de evento y lote son indeterminadas hasta que se ejecuta el grafo.

## Distribuciones escalares

Como señalamos anteriormente, un objeto `Distribution` tiene formas definidas de eventos y lotes. Comenzaremos con una utilidad para describir distribuciones:

In [ ]:
def describe_distributions(distributions):
  print('\n'.join([str(d) for d in distributions]))

En esta sección, exploraremos distribuciones *escalares*: distribuciones con una forma de evento de `[]`. Un ejemplo típico es la distribución de Poisson, especificada por una `rate`:

In [ ]:
poisson_distributions = [
    tfd.Poisson(rate=1., name='One Poisson Scalar Batch'),
    tfd.Poisson(rate=[1., 10., 100.], name='Three Poissons'),
    tfd.Poisson(rate=[[1., 10., 100.,], [2., 20., 200.]],
                name='Two-by-Three Poissons'),
    tfd.Poisson(rate=[1.], name='One Poisson Vector Batch'),
    tfd.Poisson(rate=[[1.]], name='One Poisson Expanded Batch')
]

describe_distributions(poisson_distributions)

tfp.distributions.Poisson("One_Poisson_Scalar_Batch", batch_shape=[], event_shape=[], dtype=float32)
tfp.distributions.Poisson("Three_Poissons", batch_shape=[3], event_shape=[], dtype=float32)
tfp.distributions.Poisson("Two_by_Three_Poissons", batch_shape=[2, 3], event_shape=[], dtype=float32)
tfp.distributions.Poisson("One_Poisson_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
tfp.distributions.Poisson("One_Poisson_Expanded_Batch", batch_shape=[1, 1], event_shape=[], dtype=float32)


La distribución de Poisson es una distribución escalar, por lo que la forma de su evento es siempre `[]`. Si especificamos más tarifas, estas aparecerán en forma de lote. El último par de ejemplos es interesante: solo hay una tasa única, pero debido a que esa tasa está incorporada en un arreglo numpy con una forma no vacía, esa forma se convierte en la forma de lote.

La distribución Normal estándar también es escalar. Su forma de evento es `[]`, igual que la de Poisson, pero jugaremos con ella para ver nuestro primer ejemplo de *transmisión*. La Normal se especifica mediante los parámetros `loc` y `scale`:

In [ ]:
normal_distributions = [
    tfd.Normal(loc=0., scale=1., name='Standard'),
    tfd.Normal(loc=[0.], scale=1., name='Standard Vector Batch'),
    tfd.Normal(loc=[0., 1., 2., 3.], scale=1., name='Different Locs'),
    tfd.Normal(loc=[0., 1., 2., 3.], scale=[[1.], [5.]],
               name='Broadcasting Scale')
]

describe_distributions(normal_distributions)

tfp.distributions.Normal("Standard", batch_shape=[], event_shape=[], dtype=float32)
tfp.distributions.Normal("Standard_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
tfp.distributions.Normal("Different_Locs", batch_shape=[4], event_shape=[], dtype=float32)
tfp.distributions.Normal("Broadcasting_Scale", batch_shape=[2, 4], event_shape=[], dtype=float32)


El interesante ejemplo anterior es la distribución `Broadcasting Scale`. El parámetro `loc` tiene forma `[4]` y el parámetro `scale` tiene forma `[2, 1]`. Si usamos [las reglas de difusión de Numpy](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html), la forma de lote es `[2, 4]`. Una forma equivalente (pero menos elegante y no recomendada) de definir la distribución `"Broadcasting Scale"` sería la siguiente:

In [ ]:
describe_distributions(
    [tfd.Normal(loc=[[0., 1., 2., 3], [0., 1., 2., 3.]],
                scale=[[1., 1., 1., 1.], [5., 5., 5., 5.]])])

tfp.distributions.Normal("Normal", batch_shape=[2, 4], event_shape=[], dtype=float32)


Podemos ver por qué la notación de difusión es útil, aunque también es una fuente de dolores de cabeza y errores.

### Muestreo de distribuciones escalares

Hay dos cosas principales que podemos hacer con las distribuciones: podemos ejecutar `sample` y calcular `log_prob`. Exploremos primero el muestreo. La regla básica es que cuando tomamos muestras de una distribución, el tensor resultante tiene la forma `[sample_shape, batch_shape, event_shape]`, donde `batch_shape` y `event_shape` son proporcionadas por el objeto `Distribution`, y `sample_shape` es proporcionada por la llamada a `sample`. Para distribuciones escalares, `event_shape = []`, por lo que el tensor devuelto por la muestra tendrá la forma `[sample_shape, batch_shape]`. Pongámoslo a prueba:

In [ ]:
def describe_sample_tensor_shape(sample_shape, distribution):
    print('Sample shape:', sample_shape)
    print('Returned sample tensor shape:',
          distribution.sample(sample_shape).shape)

def describe_sample_tensor_shapes(distributions, sample_shapes):
    started = False
    for distribution in distributions:
      print(distribution)
      for sample_shape in sample_shapes:
        describe_sample_tensor_shape(sample_shape, distribution)
      print()

sample_shapes = [1, 2, [1, 5], [3, 4, 5]]
describe_sample_tensor_shapes(poisson_distributions, sample_shapes)

tfp.distributions.Poisson("One_Poisson_Scalar_Batch", batch_shape=[], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1,)
Sample shape: 2
Returned sample tensor shape: (2,)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5)

tfp.distributions.Poisson("Three_Poissons", batch_shape=[3], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 3)
Sample shape: 2
Returned sample tensor shape: (2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 3)

tfp.distributions.Poisson("Two_by_Three_Poissons", batch_shape=[2, 3], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 2, 3)

In [ ]:
describe_sample_tensor_shapes(normal_distributions, sample_shapes)

tfp.distributions.Normal("Standard", batch_shape=[], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1,)
Sample shape: 2
Returned sample tensor shape: (2,)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5)

tfp.distributions.Normal("Standard_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 1)
Sample shape: 2
Returned sample tensor shape: (2, 1)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 1)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 1)

tfp.distributions.Normal("Different_Locs", batch_shape=[4], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 4)
Sample shape: 2
Returned sample tensor shape: (2, 4)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 4)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 4)

tfp.distributions.Normal("Broadc

Eso es todo lo que hay que decir sobre `sample`: los tensores de muestra devueltos tienen forma `[sample_shape, batch_shape, event_shape]`.

### Cálculo de `log_prob` para distribuciones escalares

Ahora echemos un vistazo a `log_prob`, que es algo más complicado. `log_prob` toma como entrada un tensor (no vacío) que representa las ubicaciones en las que calcular `log_prob` para la distribución. En el caso más sencillo, este tensor tendrá una forma de la forma `[sample_shape, batch_shape, event_shape]`, donde `batch_shape` y `event_shape` coinciden con las formas de lote y evento de la distribución. Recuerde una vez más que para distribuciones escalares, `event_shape = []`, por lo que el tensor de entrada tiene forma `[sample_shape, batch_shape]` En este caso, obtenemos un tensor de forma `[sample_shape, batch_shape]`:

In [ ]:
three_poissons = tfd.Poisson(rate=[1., 10., 100.], name='Three Poissons')
three_poissons

<tfp.distributions.Poisson 'Three_Poissons' batch_shape=[3] event_shape=[] dtype=float32>

In [ ]:
three_poissons.log_prob([[1., 10., 100.], [100., 10., 1]])  # sample_shape is [2].

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -2.0785608,   -3.2223587],
       [-364.73938  ,   -2.0785608,  -95.39484  ]], dtype=float32)>

In [ ]:
three_poissons.log_prob([[[[1., 10., 100.], [100., 10., 1.]]]])  # sample_shape is [1, 1, 2].

<tf.Tensor: shape=(1, 1, 2, 3), dtype=float32, numpy=
array([[[[  -1.       ,   -2.0785608,   -3.2223587],
         [-364.73938  ,   -2.0785608,  -95.39484  ]]]], dtype=float32)>

Observe cómo en el primer ejemplo, la entrada y la salida tienen forma `[2, 3]` y en el segundo ejemplo tienen forma `[1, 1, 2, 3]`.

Eso sería todo lo que habría que decir, si no fuera por la difusión. Estas son las reglas una vez que tomamos en cuenta la difusión. Lo describimos con total generalidad y observamos simplificaciones para distribuciones escalares:

1. Defina `n = len(batch_shape) + len(event_shape)`. (Para distribuciones escalares, `len(event_shape)=0`.)
2. Si el tensor de entrada `t` tiene menos de `n` dimensiones, rellene su forma mediante la incorporación de dimensiones de tamaño `1` a la izquierda hasta que tenga exactamente `n` dimensiones. Llame al tensor `t'` resultante.
3. Difunda las `n` dimensiones más a la derecha de `t'` contra `[batch_shape, event_shape]` de la distribución para la que se está calculando una `log_prob`. Más detalladamente: para las dimensiones donde `t'` ya coincide con la distribución, no haga nada, y para las dimensiones donde `t'` tiene un singleton, replique ese singleton el número de veces correspondiente. Cualquier otra situación es un error. (Para distribuciones escalares, solo difundimos contra `batch_shape`, ya que event_shape = `[]`).
4. Ahora finalmente podemos calcular `log_prob`. El tensor resultante tendrá forma `[sample_shape, batch_shape]`, donde `sample_shape` se define como cualquier dimensión de `t` o `t'` a la izquierda de las `n` dimensiones más a la derecha: `sample_shape = shape(t)[:-n]`.

Esto puede ser un desastre si no sabe lo que significa, así que trabajemos con algunos ejemplos:

In [ ]:
three_poissons.log_prob([10.])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-16.104412 ,  -2.0785608, -69.05272  ], dtype=float32)>

El tensor `[10.]` (con forma `[1]`) se difunde a través de `batch_shape` de 3, por lo que evaluamos la probabilidad logarítmica de los tres Poisson en el valor 10.

In [ ]:
three_poissons.log_prob([[[1.], [10.]], [[100.], [1000.]]])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[-1.0000000e+00, -7.6974149e+00, -9.5394836e+01],
        [-1.6104412e+01, -2.0785608e+00, -6.9052719e+01]],

       [[-3.6473938e+02, -1.4348087e+02, -3.2223587e+00],
        [-5.9131279e+03, -3.6195427e+03, -1.4069575e+03]]], dtype=float32)>

En el ejemplo anterior, el tensor de entrada tiene forma `[2, 2, 1]`, mientras que el objeto de distribuciones tiene una forma de lote de 3. Entonces, para cada una de las dimensiones de muestra `[2, 2]`, el valor único proporcionado obtiene difusiones a cada uno de los tres Poisson.

Una forma posiblemente útil de pensarlo: debido a que `three_poissons` tiene `batch_shape = [2, 3]`, una llamada a `log_prob` debe tomar un tensor cuya última dimensión sea 1 o 3; cualquier otra cosa es un error. (Las reglas de difusión de numpy tratan el caso especial de un escalar como totalmente equivalente a un tensor de forma `[1]`).

Pongamos a prueba nuestras habilidades experimentando con la distribución de Poisson más compleja con `batch_shape = [2, 3]`:

In [ ]:
poisson_2_by_3 = tfd.Poisson(
    rate=[[1., 10., 100.,], [2., 20., 200.]],
    name='Two-by-Three Poissons')

In [ ]:
poisson_2_by_3.log_prob(1.)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([1.])  # Exactly equivalent to above, demonstrating the scalar special case.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([[1., 1., 1.], [1., 1., 1.]])  # Another way to write the same thing. No broadcasting.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([[1., 10., 100.]])  # Input is [1, 3] broadcast to [2, 3].

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ -1.       ,  -2.0785608,  -3.2223587],
       [ -1.3068528,  -5.14709  , -33.90767  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([[1., 10., 100.], [1., 10., 100.]])  # Equivalent to above. No broadcasting.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ -1.       ,  -2.0785608,  -3.2223587],
       [ -1.3068528,  -5.14709  , -33.90767  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([[1., 1., 1.], [2., 2., 2.]])  # No broadcasting.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -14.701683 , -190.09653  ]], dtype=float32)>

In [ ]:
poisson_2_by_3.log_prob([[1.], [2.]])  # Equivalent to above. Input shape [2, 1] broadcast to [2, 3].

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -14.701683 , -190.09653  ]], dtype=float32)>

Los ejemplos anteriores implicaban la difusión del lote, pero la forma de la muestra estaba vacía. Supongamos que tenemos una colección de valores y queremos obtener la probabilidad logarítmica de cada valor en cada punto del lote. Podríamos hacerlo manualmente:

In [ ]:
poisson_2_by_3.log_prob([[[1., 1., 1.], [1., 1., 1.]], [[2., 2., 2.], [2., 2., 2.]]])  # Input shape [2, 2, 3].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

O podríamos dejar que la difusión se encargue de la última dimensión del lote:

In [ ]:
poisson_2_by_3.log_prob([[[1.], [1.]], [[2.], [2.]]])  # Input shape [2, 2, 1].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

También podríamos (aunque quizás no resulte tan natural) dejar que la difusión maneje solo la primera dimensión del lote:

In [ ]:
poisson_2_by_3.log_prob([[[1., 1., 1.]], [[2., 2., 2.]]])  # Input shape [2, 1, 3].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

O podríamos dejar que la difusión maneje *ambas* dimensiones del lote:

In [ ]:
poisson_2_by_3.log_prob([[[1.]], [[2.]]])  # Input shape [2, 1, 1].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

Lo que acabamos de explicar funcionaba bien cuando solo queríamos dos valores, pero supongamos que tuviéramos una larga lista de valores y quisiéramos evaluarlos en cada punto del lote. Para eso, la siguiente notación, que agrega dimensiones adicionales de tamaño 1 al lado derecho de la forma, es extremadamente útil:

In [ ]:
poisson_2_by_3.log_prob(tf.constant([1., 2.])[..., tf.newaxis, tf.newaxis])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

Este es un ejemplo de [notación de segmento con pasos](https://www.tensorflow.org/api_docs/cc/class/tensorflow/ops/strided-slice), que vale la pena conocer.

Si retomamos `three_poissons` para completar, el mismo ejemplo se ve así:

In [ ]:
three_poissons.log_prob([[1.], [10.], [50.], [100.]])

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [ -16.104412 ,   -2.0785608,  -69.05272  ],
       [-149.47777  ,  -43.34851  ,  -18.219261 ],
       [-364.73938  , -143.48087  ,   -3.2223587]], dtype=float32)>

In [ ]:
three_poissons.log_prob(tf.constant([1., 10., 50., 100.])[..., tf.newaxis])  # Equivalent to above.

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [ -16.104412 ,   -2.0785608,  -69.05272  ],
       [-149.47777  ,  -43.34851  ,  -18.219261 ],
       [-364.73938  , -143.48087  ,   -3.2223587]], dtype=float32)>

## Distribuciones multivariadas

Pasemos ahora a las distribuciones multivariadas, que tienen forma de evento no vacía. Veamos distribuciones multinomiales.

In [ ]:
multinomial_distributions = [
    # Multinomial is a vector-valued distribution: if we have k classes,
    # an individual sample from the distribution has k values in it, so the
    # event_shape is `[k]`.
    tfd.Multinomial(total_count=100., probs=[.5, .4, .1],
                    name='One Multinomial'),
    tfd.Multinomial(total_count=[100., 1000.], probs=[.5, .4, .1],
                    name='Two Multinomials Same Probs'),
    tfd.Multinomial(total_count=100., probs=[[.5, .4, .1], [.1, .2, .7]],
                    name='Two Multinomials Same Counts'),
    tfd.Multinomial(total_count=[100., 1000.],
                    probs=[[.5, .4, .1], [.1, .2, .7]],
                    name='Two Multinomials Different Everything')

]

describe_distributions(multinomial_distributions)

tfp.distributions.Multinomial("One_Multinomial", batch_shape=[], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Same_Probs", batch_shape=[2], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Same_Counts", batch_shape=[2], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Different_Everything", batch_shape=[2], event_shape=[3], dtype=float32)


Observe cómo en los últimos tres ejemplos, batch_shape es siempre `[2]`, pero podemos usar la difusión para tener un `total_count` compartido o `probs` compartidas (o ninguno de los dos), porque en el fondo se difunden para que tengan la misma forma.

El muestreo es sencillo, dado lo que ya sabemos:

In [ ]:
describe_sample_tensor_shapes(multinomial_distributions, sample_shapes)

tfp.distributions.Multinomial("One_Multinomial", batch_shape=[], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 3)
Sample shape: 2
Returned sample tensor shape: (2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 3)

tfp.distributions.Multinomial("Two_Multinomials_Same_Probs", batch_shape=[2], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 2, 3)

tfp.distributions.Multinomial("Two_Multinomials_Same_Counts", batch_shape=[2], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]


El cálculo de las probabilidades logarítmicas es igualmente sencillo. Trabajemos un ejemplo con distribuciones Normales Multivariadas diagonales. (Las multinomiales no se llevan muy bien con la difusión, ya que las restricciones en los recuentos y las probabilidades implican que la difusión a menudo producirá valores inadmisibles). Usaremos un lote de 2 distribuciones tridimensionales con la misma media, pero diferentes escalas (desviaciones estándar):

In [ ]:
two_multivariate_normals = tfd.MultivariateNormalDiag(loc=[1., 2., 3.], scale_diag=tf.ones([2, 3]) * [[1.], [2.]])
two_multivariate_normals

<tfp.distributions.MultivariateNormalDiag 'MultivariateNormalDiag' batch_shape=[2] event_shape=[3] dtype=float32>

Ahora evaluemos la probabilidad logarítmica de cada punto del lote en su media y en una media desplazada:

In [ ]:
two_multivariate_normals.log_prob([[[1., 2., 3.]], [[3., 4., 5.]]])  # Input has shape [2,1,3].

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.7568154, -4.836257 ],
       [-8.756816 , -6.336257 ]], dtype=float32)>

De manera exactamente equivalente, podemos usar [https://www.tensorflow.org/api_docs/cc/class/tensorflow/ops/strided-slice](notación de segmento con pasos) para insertar una dimensión extra de forma=1 en medio de una constante:

In [ ]:
two_multivariate_normals.log_prob(
    tf.constant([[1., 2., 3.], [3., 4., 5.]])[:, tf.newaxis, :])  # Equivalent to above.

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.7568154, -4.836257 ],
       [-8.756816 , -6.336257 ]], dtype=float32)>

Por otro lado, si no insertamos la dimensión adicional, pasamos `[1., 2., 3.]` al primer punto del lote y `[3., 4., 5.]` al segundo:

In [ ]:
two_multivariate_normals.log_prob(tf.constant([[1., 2., 3.], [3., 4., 5.]]))

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.7568154, -6.336257 ], dtype=float32)>

## Técnicas de manipulación de formas

### El biyector Reshape

El biyector `Reshape` sirve para remodelar la *event_shape* de una distribución. Veamos un ejemplo:

In [ ]:
six_way_multinomial = tfd.Multinomial(total_count=1000., probs=[.3, .25, .2, .15, .08, .02])
six_way_multinomial

<tfp.distributions.Multinomial 'Multinomial' batch_shape=[] event_shape=[6] dtype=float32>

Creamos una multinomial con una forma de evento de `[6]`. El biyector Reshape nos permite tratar esto como una distribución con una forma de evento de `[2, 3]`.

Un `Bijector` representa una función diferenciable uno a uno en un subconjunto abierto de ${\mathbb R}^n$. `Bijectors` se usan junto con `TransformedDistribution`, que modela una distribución $p(y)$ en términos de una distribución base $p(x)$ y un `Bijector` que representa $Y = g(X)$. Veámoslo en acción:

In [ ]:
transformed_multinomial = tfd.TransformedDistribution(
    distribution=six_way_multinomial,
    bijector=tfb.Reshape(event_shape_out=[2, 3]))
transformed_multinomial

<tfp.distributions.TransformedDistribution 'reshapeMultinomial' batch_shape=[] event_shape=[2, 3] dtype=float32>

In [ ]:
six_way_multinomial.log_prob([500., 100., 100., 150., 100., 50.])

<tf.Tensor: shape=(), dtype=float32, numpy=-178.21973>

In [ ]:
transformed_multinomial.log_prob([[500., 100., 100.], [150., 100., 50.]])

<tf.Tensor: shape=(), dtype=float32, numpy=-178.21973>

Esto es lo *único* que puede hacer el biyector `Reshape`: no puede convertir dimensiones de eventos en dimensiones de lotes ni viceversa.

### La distribución Independent

La distribución `Independent` sirve para tratar una colección de distribuciones independientes, no necesariamente idénticas (también conocidas como un lote de) como una distribución única. De manera más concisa, `Independent` permite convertir dimensiones en `batch_shape` en dimensiones en `event_shape`. Lo ilustraremos con el ejemplo:

In [ ]:
two_by_five_bernoulli = tfd.Bernoulli(
    probs=[[.05, .1, .15, .2, .25], [.3, .35, .4, .45, .5]],
    name="Two By Five Bernoulli")
two_by_five_bernoulli

<tfp.distributions.Bernoulli 'Two_By_Five_Bernoulli' batch_shape=[2, 5] event_shape=[] dtype=int32>

Podemos pensar en esto como un arreglo de monedas de dos por cinco con las probabilidades de cara asociadas. Evaluemos la probabilidad de un conjunto particular y arbitrario de unos y ceros:

In [ ]:
pattern = [[1., 0., 0., 1., 0.], [0., 0., 1., 1., 1.]]
two_by_five_bernoulli.log_prob(pattern)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[-2.9957323 , -0.10536051, -0.16251892, -1.609438  , -0.2876821 ],
       [-0.35667497, -0.4307829 , -0.91629076, -0.79850775, -0.6931472 ]],
      dtype=float32)>

Podemos utilizar `Independent` para convertir esto en dos "conjuntos de Bernoulli de cinco" diferentes, lo cual es útil si queremos considerar una "fila" de lanzamientos de monedas que aparecen en un patrón determinado como un resultado único:

In [ ]:
two_sets_of_five = tfd.Independent(
    distribution=two_by_five_bernoulli,
    reinterpreted_batch_ndims=1,
    name="Two Sets Of Five")
two_sets_of_five

<tfp.distributions.Independent 'Two_Sets_Of_Five' batch_shape=[2] event_shape=[5] dtype=int32>

Matemáticamente, estamos calculando la probabilidad logarítmica de cada "conjunto" de cinco al sumar las probabilidades logarítmicas de los cinco lanzamientos de moneda "independientes" del conjunto, que es de donde la distribución obtiene su nombre:

In [ ]:
two_sets_of_five.log_prob(pattern)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-5.160732 , -3.1954036], dtype=float32)>

Podemos ir aún más lejos y usar `Independent` para crear una distribución donde los eventos individuales sean un conjunto de Bernoulli de dos por cinco:

In [ ]:
one_set_of_two_by_five = tfd.Independent(
    distribution=two_by_five_bernoulli, reinterpreted_batch_ndims=2,
    name="One Set Of Two By Five")
one_set_of_two_by_five.log_prob(pattern)

<tf.Tensor: shape=(), dtype=float32, numpy=-8.356134>

Vale la pena señalar que desde la perspectiva de `sample` el uso de `Independent` no cambia nada:

In [ ]:
describe_sample_tensor_shapes(
    [two_by_five_bernoulli,
     two_sets_of_five,
     one_set_of_two_by_five],
    [[3, 5]])

tfp.distributions.Bernoulli("Two_By_Five_Bernoulli", batch_shape=[2, 5], event_shape=[], dtype=int32)
Sample shape: [3, 5]
Returned sample tensor shape: (3, 5, 2, 5)

tfp.distributions.Independent("Two_Sets_Of_Five", batch_shape=[2], event_shape=[5], dtype=int32)
Sample shape: [3, 5]
Returned sample tensor shape: (3, 5, 2, 5)

tfp.distributions.Independent("One_Set_Of_Two_By_Five", batch_shape=[], event_shape=[2, 5], dtype=int32)
Sample shape: [3, 5]
Returned sample tensor shape: (3, 5, 2, 5)



Para terminar, sugerimos que el lector analice las diferencias y similitudes entre un lote vectorial de distribuciones `Normal` y una distribución `MultivariateNormalDiag` desde una perspectiva de muestreo y probabilidad logarítmica. ¿Cómo podemos usar `Independent` para construir una `MultivariateNormalDiag` a partir de un lote de mensajes `Normal`? (Tenga en cuenta que `MultivariateNormalDiag` en realidad no se implementa de esta manera).